## Challenge

Las preguntas son las siguientes: 
1.	¿Qué cervecería produce la cerveza más fuerte según ABV? 
2.	¿Cuál de los factores (aroma, taste, apperance, palette) es más importante para determinar la calidad general de una cerveza? 
3.	¿Si tuviera que elegir 3 cervezas para recomendar usando sólo estos datos, cuáles elegiría? 

4.	¿Si yo típicamente disfruto una cerveza debido a su aroma y apariencia, qué estilo de cerveza debería probar? 

__Nota:__ Cambie de orden la pregunta 2. con la 3. (tal como me la enviaron), pues uso los resultados de la pregunta 2. para resolver la 3.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Inspeccionando el dataset

In [2]:
!ls

LICENSE          README.md        beer_reviews.csv challenge.ipynb


In [3]:
df = pd.read_csv("beer_reviews.csv")

In [4]:
df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [5]:
print("Cantidad de datos:",df.shape[0])
print("Cantidad de columnas:",df.shape[1])

Cantidad de datos: 1586614
Cantidad de columnas: 13


In [6]:
# dtypes of attributes
df.dtypes

brewery_id              int64
brewery_name           object
review_time             int64
review_overall        float64
review_aroma          float64
review_appearance     float64
review_profilename     object
beer_style             object
review_palate         float64
review_taste          float64
beer_name              object
beer_abv              float64
beer_beerid             int64
dtype: object

Notamos que hay cervezas que se encuentran repetidas, es decir, tienen más de un registro hecho por distintos _reviewers_ y en distintos tiempos.

In [36]:
# cantidad de tipos de cervezas
df["beer_name"].value_counts()

90 Minute IPA                                   3290
India Pale Ale                                  3130
Old Rasputin Russian Imperial Stout             3111
Sierra Nevada Celebration Ale                   3000
Two Hearted Ale                                 2728
Arrogant Bastard Ale                            2704
Stone Ruination IPA                             2704
Sierra Nevada Pale Ale                          2587
Stone IPA (India Pale Ale)                      2575
Pliny The Elder                                 2527
Founders Breakfast Stout                        2502
Pale Ale                                        2500
Sierra Nevada Bigfoot Barleywine Style Ale      2492
La Fin Du Monde                                 2483
60 Minute IPA                                   2475
Storm King Stout                                2452
Duvel                                           2450
Brooklyn Black Chocolate Stout                  2447
Bell's Hopslam Ale                            

### Pregunta 1

__¿Qué cervecería produce la cerveza más fuerte según ABV?__

La opción más obvia sería ordenar el `dataframe` por la columna `beer_abv`. Sin embargo dado la gran cantidad de datos sería muy ineficiente. En cambio se iterará sobre el `dataframe` guardando datos de las $10$ cervezas más fuertes: 

In [39]:
df1 = df[["beer_abv", "brewery_name", "beer_style", "beer_name", "beer_beerid"]]
df1.head()

,beer_abv,brewery_name,beer_style,beer_name,beer_beerid
0,5.0,Vecchio Birraio,Hefeweizen,Sausa Weizen,47986
1,6.2,Vecchio Birraio,English Strong Ale,Red Moon,48213
2,6.5,Vecchio Birraio,Foreign / Export Stout,Black Horse Black Beer,48215
3,5.0,Vecchio Birraio,German Pilsener,Sausa Pils,47969
4,7.7,Caldera Brewing Company,American Double / Imperial IPA,Cauldron DIPA,64883


In [40]:
n = 10 # n cervezas mas fuertes
max_abv_data = [(-float("inf"),None,None,None,None)]*n
beer_ids = []

for index,row in df1.iterrows():
    if row["beer_abv"] > max_abv_data[-1][0]:
        if row["beer_beerid"] in [x[-1] for x in max_abv_data]:
            continue # no contar cervezas repetidas
        max_abv_data.append( (row["beer_abv"], row["brewery_name"], row["beer_style"], row["beer_name"], row["beer_beerid"]))
        max_abv_data.sort(reverse=True)
        max_abv_data = max_abv_data[:n]

In [42]:
for br_abv,bw_name,br_style,br_name,_ in max_abv_data:
    print ("{0} - {1} - {2} - {3}".format(bw_name,br_name,br_style,br_abv))

Schorschbräu - Schorschbräu Schorschbock 57% - Eisbock - 57.7
Schorschbräu - Schorschbräu Schorschbock 43% - Eisbock - 43.0
BrewDog - Sink The Bismarck! - American Double / Imperial IPA - 41.0
Schorschbräu - Schorschbräu Schorschbock 40% - Eisbock - 39.44
De Struise Brouwers - Black Damnation VI - Messy - American Double / Imperial Stout - 39.0
BrewDog - Tactical Nuclear Penguin - American Double / Imperial Stout - 32.0
Schorschbräu - Schorschbräu Schorschbock 31% - Eisbock - 30.86
Hair of the Dog Brewing Company / Brewery and Tasting Room - Dave - English Barleywine - 29.0
BrewDog - Ghost Deer - Belgian Strong Pale Ale - 28.0
Boston Beer Company (Samuel Adams) - Samuel Adams Utopias - American Strong Ale - 27.0


De los resultados se ve que la cervecería _Schorschbräu_ produce las dos cervezas más fuertes. Por lo tanto se infiere que es está cervecería la que produce la cerveza más fuerte según ABV en el dataset.

### Pregunta 2

__¿Cuál de los factores (aroma, taste, apperance, palette) es más importante para determinar la calidad general de una cerveza?__

In [80]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

La estrategia a seguir será la siguiente: Se supondrá que existe una relación lineal entre las $4$ variables de interés y el `review_overall`. Se entrenarán $4$ regresores lineales (en un respectivo _training set_), en cada uno omitiendo una de las variables. 

Se calculará el _score_ (coeficiente de determinación) de cada regresor obtenido (sobre un _testing set_). Cuanto más bajo sea el _score_, esto indicará que la variable que estamos omitiendo es importante más importante. Bajo dicho criterio se determinará la importancia de cada variable.


In [81]:
y = df["review_overall"].as_matrix()
X = df.as_matrix(columns=["review_aroma", "review_appearance", "review_palate", "review_taste"])

# Se divide training y testing en 75% y 25% respectivamente
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [87]:
lr1 = LinearRegression()
# excluding review_taste
lr1.fit(X_train[:,[0,1,2]],y_train)
print("Coefficient of determination:", lr1.score(X_test[:,[0,1,2]], y_test))


Coefficient of determination: 0.5444870660229477


In [88]:
lr2 = LinearRegression()
# excluding review_palate
lr2.fit(X_train[:,[0,1,3]],y_train)
print("Coefficient of determination:", lr2.score(X_test[:,[0,1,3]], y_test))

Coefficient of determination: 0.6285849919483448


In [90]:
lr3 = LinearRegression()
# excluding review_appearance
lr3.fit(X_train[:,[0,2,3]],y_train)
print("Coefficient of determination:", lr3.score(X_test[:,[0,2,3]], y_test))

Coefficient of determination: 0.6531073161147438


In [91]:
lr4 = LinearRegression()
# excluding review_aroma
lr4.fit(X_train[:,[1,2,3]],y_train)
print("Coefficient of determination:", lr4.score(X_test[:,[1,2,3]], y_test))

Coefficient of determination: 0.6527652817980704


Los resultados indican que las variables más importantes para determinar la calidad `overall` de la cerveza es (de mayor a menor):
1. `review_taste`
2. `review_palate`
3. `review_aroma`
4. `review_appearance`

### Pregunta 3

__¿Si tuviera que elegir 3 cervezas para recomendar usando sólo estos datos, cuáles elegiría?__

En este pregunta, a primer vista, el atributo de mayor importancia es `review_overall`. Sin embargo también es importante fijarse en el tiempo en el que se hizo el review. Como se puede ver en la celda de abajo, cada cerveza puede tener más de un _review_ en distintos tiempos. Para medir la calidad de una cerveza, se tomará en cuenta _su historia_, vale decir se sacará un promedio de los distintos `review_overall` para cada cerveza.

In [137]:
# reviews de cada cervezas
df["beer_name"].value_counts()

90 Minute IPA                                   3290
India Pale Ale                                  3130
Old Rasputin Russian Imperial Stout             3111
Sierra Nevada Celebration Ale                   3000
Two Hearted Ale                                 2728
Arrogant Bastard Ale                            2704
Stone Ruination IPA                             2704
Sierra Nevada Pale Ale                          2587
Stone IPA (India Pale Ale)                      2575
Pliny The Elder                                 2527
Founders Breakfast Stout                        2502
Pale Ale                                        2500
Sierra Nevada Bigfoot Barleywine Style Ale      2492
La Fin Du Monde                                 2483
60 Minute IPA                                   2475
Storm King Stout                                2452
Duvel                                           2450
Brooklyn Black Chocolate Stout                  2447
Bell's Hopslam Ale                            

Se extraen y guardan los datos de interés en el diccionario `beer_map`:

In [138]:
beer_map = dict()

for index,row in df.iterrows():
    if row["beer_name"] not in beer_map:
        beer_map[row["beer_name"]] = [row["review_overall"]]
    else:
        beer_map[row["beer_name"]].append( row["review_overall"] )

for key,ll in beer_map.items():
    beer_map[key] = sum(ll)/len(ll)

Se busca en el diccionario anterior las $N$ cervezas con mayor `review_overall`

In [142]:
n = 20 # probar aumentando el n
top_beers = [ (-float("inf"),None) ]*n

for key,overall in beer_map.items():
    if overall > top_beers[-1][0]:
        top_beers.append( (overall,key) )
        top_beers.sort(reverse=True)
        top_beers = top_beers[:n]

In [143]:
top_beers

[(5.0, 'Triple OOO'),
 (5.0, 'Suicide By Hops'),
 (5.0, 'Southern Belle'),
 (5.0, 'Schwindel Alt'),
 (5.0, 'Replic Ale (2010)'),
 (5.0, "Quinn's Marathon Mild"),
 (5.0, 'Quaker Oatmeal Stout'),
 (5.0, 'Pike Old 89 Nitro Brown'),
 (5.0, 'One Hop Wonder Version 12'),
 (5.0, 'Oak Aged Los Diablos Del Paso'),
 (5.0, 'Limping Mallard'),
 (5.0, 'Javaspresso Nitro Stout'),
 (5.0, 'Hops Bandit'),
 (5.0, "Gene's American Brown Ale"),
 (5.0, 'Final Countdown'),
 (5.0, 'Cauldron Brew'),
 (5.0, 'Calapooia Barleywine'),
 (5.0, 'BourbonDog ESB'),
 (5.0, 'Bigwood Oak Aged Stout'),
 (5.0, "Ackerman's Imperial Double Stout (Winterfest Replicale 2011)")]

__Problema!__ hay demasiadas cervezas con `review_overall=5` y por lo tanto no fue un buen criterio para hacer una recomendación.

Se necesitan agregar más criterios para ordenar (para desempatar!), y para ello se ocuparán los resultados de la pregunta anterior. Vale decir, se determinarán las $N$ mejores cervezas ordenand según:
1. `review_overall`
2. `review_taste`
3. `review_palate`
4. `review_aroma`
5. `review_appearance`
6. Cantidad de reviews.

Vale decir, si hay empate en `review_overall` se decide por `review_taste`. Si hay empate en  `review_taste` se decide por `review_palate` y así sucesivamente. 

Al final, si hay empates en todos estos criterios, se definirá por la cantidad de reviews que tiene cada cerveza. Esto tiene sentido, pues mientras más reviews, más "robustos" son los otros indicadores. 


La implementación es análoga a la anterior:

In [117]:
# esto podria hacerse mejor...
bm_overall = dict()
bm_taste = dict()
bm_palate = dict()
bm_aroma = dict()
bm_appearance = dict()

for index,r in df.iterrows():
    key = r["beer_name"]
    if row["beer_name"] not in beer_map:
        bm_overall[key] = [r["review_overall"]]
        bm_taste[key] = [r["review_taste"]]
        bm_palate[key] = [r["review_palate"]]
        bm_aroma[key] = [r["review_aroma"]]
        bm_appearance[key] = [r["review_appearance"]]
    else:
        bm_overall[key].append( r["review_overall"] )
        bm_taste[key].append( r["review_taste"] )
        bm_palate[key].append( r["review_palate"] )
        bm_aroma[key].append( r["review_aroma"] )
        bm_appearance[key].append( r["review_appearance"] )
        
beer_map = dict()        
for key in bm_overall.keys():
    k = len(bm_overall[key])
    beer_map[key] = (sum(bm_overall[key])/k,
                     sum(bm_taste[key])/k,
                     sum(bm_palate[key])/k,
                     sum(bm_aroma[key])/k,
                     sum(bm_appearance[key])/k,
                     k)

Se encuentran las $N$ mejores cervezas bajo estos criterios:

In [122]:
n = 20 # probar aumentando el n
top_beers = [ (-float("inf"),-float("inf"),-float("inf"),
               -float("inf"),-float("inf"),-float("inf"),None) ]*n

for key,reviews in beer_map.items():
    name = key
    if reviews > top_beers[-1][:5]:
        # se comparan uno a unos los criterios
        top_beers.append( (*reviews,name) )
        top_beers.sort(reverse=True)
        top_beers = top_beers[:n]

In [123]:
top_beers

[(5.0, 5.0, 5.0, 5.0, 5.0, 1, "Zebulon's Peated Porter"),
 (5.0, 5.0, 5.0, 5.0, 5.0, 1, 'Zatte'),
 (5.0, 5.0, 5.0, 5.0, 5.0, 1, "Young's Bitter"),
 (5.0, 5.0, 5.0, 5.0, 5.0, 1, 'Xyauyù Etichetta Rame (Copper)'),
 (5.0, 5.0, 5.0, 5.0, 5.0, 1, 'Wolnzacher Oktoberfest-Bier'),
 (5.0, 5.0, 5.0, 5.0, 5.0, 1, 'Witches Brew Doppelbock'),
 (5.0, 5.0, 5.0, 5.0, 5.0, 1, 'Westmalle Trappist Tripel'),
 (5.0, 5.0, 5.0, 5.0, 5.0, 1, 'Victory At Sea (Barrel Aged) Imperial Porter'),
 (5.0, 5.0, 5.0, 5.0, 5.0, 1, 'Triticum Bock'),
 (5.0, 5.0, 5.0, 5.0, 5.0, 1, 'The Ugly American'),
 (5.0, 5.0, 5.0, 5.0, 5.0, 1, 'The Crusher'),
 (5.0, 5.0, 5.0, 5.0, 5.0, 1, 'Tannenzäpfle'),
 (5.0, 5.0, 5.0, 5.0, 5.0, 1, 'Strathdee Old Barleywine'),
 (5.0, 5.0, 5.0, 5.0, 5.0, 1, "Stoudt's Abbey Triple (750ml Release)"),
 (5.0, 5.0, 5.0, 5.0, 5.0, 1, 'Stone Brew'),
 (5.0, 5.0, 5.0, 5.0, 5.0, 1, 'Spanish Peaks White Ale'),
 (5.0, 5.0, 5.0, 5.0, 5.0, 1, 'Smooth India Pale Ale'),
 (5.0, 5.0, 5.0, 5.0, 5.0, 1, 'Sim City IPA'),

Como puede verse, pese a haber agregado más criterio sigue habiendo empate. Una posible alternativa para desempatar, sería solo considerar las cervezas que tengas una cantidad `n_min` de reviews (y así no tomar en cuenta las que tienen sólo $1$, que al parecer son muchas). 

Por ahora, le recomendaría a mi amigo elegir cualquiera $3$ cervezas de esta lista :-).

### Pregunta 4

__¿Si yo típicamente disfruto una cerveza debido a su aroma y apariencia, qué estilo de cerveza debería probar?__


Estrategia: Por cada tipo de cerveza se calculará su promedio en `review_aroma` y `review_apariencia`. Luego se determinará el mejor estilo, tomando en cuenta primero `review_aroma` y luego `review_appearance` (de acuerdo a los resultados de 2.).



In [124]:
# reviews de cada  cervezas
df["beer_style"].value_counts()

American IPA                           117586
American Double / Imperial IPA          85977
American Pale Ale (APA)                 63469
Russian Imperial Stout                  54129
American Double / Imperial Stout        50705
American Porter                         50477
American Amber / Red Ale                45751
Belgian Strong Dark Ale                 37743
Fruit / Vegetable Beer                  33861
American Strong Ale                     31945
Belgian Strong Pale Ale                 31490
Saison / Farmhouse Ale                  31480
American Adjunct Lager                  30749
Tripel                                  30328
Witbier                                 30140
Hefeweizen                              27908
American Barleywine                     26728
American Brown Ale                      25297
American Stout                          24538
American Pale Wheat Ale                 24204
Märzen / Oktoberfest                    23523
English Pale Ale                  

La implementación es análoga a la del problema 3:

In [126]:
sm_aroma = dict()
sm_appearance = dict()

for i,r in df.iterrows():
    key = r["beer_style"]
    if key not in sm_aroma:
        sm_aroma[key] = [r["review_aroma"]]
        sm_appearance[key] = [r["review_appearance"]]
    else:
        sm_aroma[key].append( r["review_aroma"] )
        sm_appearance[key].append( r["review_appearance"] )
        
style_map = dict()
for key in sm_aroma.keys():
    style_map[key] = (sum(sm_aroma[key])/len(sm_aroma[key]),
                      sum(sm_appearance[key])/len(sm_appearance[key]))

In [135]:
n = 5 # probar aumentando el n
top_styles = [ (-float("inf"),-float("inf"),None) ]*n

for style,reviews in style_map.items():
    if reviews > top_styles[-1][:2]:
        # se comparan uno a unos los criterios
        top_styles.append( (*reviews,style) )
        top_styles.sort(reverse=True)
        top_styles = top_styles[:n]

In [136]:
top_styles

[(4.160664628734839, 4.163632777832561, 'American Double / Imperial Stout'),
 (4.156778069846038, 3.964513706346226, 'Eisbock'),
 (4.132533451288289, 4.1179641711821295, 'Quadrupel (Quad)'),
 (4.126756209958413, 4.0054512757109135, 'American Wild Ale'),
 (4.118043087971275, 3.9124775583482942, 'Lambic - Unblended')]

Entonces, en base a los resultados, la mejor recomendación sería el estilo: __'American Double / Imperial Stout'__.